# **Pipeline**

## **1. Introduction**

Here you need to set all the necessary libraries and the data that will be used 
in the project. Also, it is important to specify all the path and all the parameters
that will be used in the project. 

In [1]:
import os
import cv2
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, "..\\Scripts")

from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from ImagePipeline import ImagePipeline
from ImageDatastore import ImageDatastore
from Utils import create_or_clear_directory
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from NeuralFeatureExtractor import MobileNetFeatureExtractor
from sklearn.metrics import accuracy_score, top_k_accuracy_score

### **Params**

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = os.cpu_count()

print("Device:", device)
print("Number of workers:", num_workers)

Device: cpu
Number of workers: 8


In [3]:
extract_features = False

type_of_train = "train"

batch_size = 512

transform = transforms.Compose(
    [
        transforms.Resize(
            (232, 232), interpolation=transforms.InterpolationMode.BILINEAR
        ),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [4]:
extractor = MobileNetFeatureExtractor()

In [5]:
preprocess_image = True
type_of_preprocess = "neural"

### **Utils**

In [6]:
def save_features(x, y, filename):
    np.save(
        os.path.join("..", "Features", "features", filename),
        x,
        allow_pickle=False,
    )
    np.save(
        os.path.join("..", "Features", "labels", filename),
        y,
        allow_pickle=False,
    )


def load_features(filename):
    x = np.load(
        os.path.join("..", "Features", "features", filename),
        allow_pickle=False,
    )
    y = np.load(
        os.path.join("..", "Features", "labels", filename),
        allow_pickle=False,
    )
    return x, y

## **2. Dataset**

Create the dataloaders and the dataset that will be used in the project.

In [7]:
train_set = ImageDatastore(type_of_train, transform=transform)
train_set_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)

In [8]:
val_set = ImageDatastore("val_set", transform=transform)
val_set_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

In [9]:
val_deg = ImageDatastore("val_degradate", transform=transform)
val_deg_loader = DataLoader(
    val_deg,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

## **3. Feature extraction**

Extract the features from the dataset only if it is necessary, using the feature extraction
methods specified in the params.

In [10]:
if extract_features:
    x_train, y_train = extractor.compute_features(train_set_loader)
    save_features(x_train, y_train, "train_features.npy")
else:
    x_train, y_train = load_features("train_features.npy")

x_train.shape, y_train.shape

((5020, 1280), (5020,))

In [11]:
if extract_features:
    x_val, y_val = extractor.compute_features(val_set_loader)
    save_features(x_val, y_val, "val_features.npy")
else:
    x_val, y_val = load_features("val_features.npy")

x_val.shape, y_val.shape

((11994, 1280), (11994,))

In [12]:
if extract_features:
    x_val_deg, y_val_deg = extractor.compute_features(val_deg_loader)
    save_features(x_val_deg, y_val_deg, "val_deg_features.npy")
else:
    x_val_deg, y_val_deg = load_features("val_deg_features.npy")

x_val_deg.shape, y_val_deg.shape

((11994, 1280), (11994,))

## **4. Models**

Use the features extracted in the previous step to train the models.

### **4.1 k-NN Classifier**

In [13]:
knn = KNeighborsClassifier(
    n_neighbors=51, n_jobs=-1, weights="distance", metric="cosine"
)
knn.fit(x_train, y_train)

KNeighborsClassifier(metric='cosine', n_jobs=-1, n_neighbors=51,
                     weights='distance')

### **4.2 Random Forest Classifier**

In [15]:
rf = RandomForestClassifier(criterion="gini", n_estimators=100, n_jobs=-1)
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

## **5. Evaluation**

Evaluate the models using the metrics specified in the params.

### **5.1 k-NN Classifier**

In [15]:
knn_pred = knn.predict(x_val)
print(f"Val: {accuracy_score(y_val, knn_pred)}")

knn_score = knn.predict_proba(x_val)
print(f"Val top 5: {top_k_accuracy_score(y_val, knn_score, k=5)}")

Val: 0.26088044022011003
Val top 5: 0.5158412539603134


### **5.2 Random Forest Classifier**

In [17]:
rf_pred = rf.predict(x_val)
print(f"Val: {accuracy_score(y_val, rf_pred)}")

rf_score = rf.predict_proba(x_val)
print(f"Val top 5: {top_k_accuracy_score(y_val, rf_score, k=5)}")

Val: 0.1520760380190095
Val top 5: 0.3133233283308321


## **6. Evaluation Degraded**

In this section, we will evaluate the models using the degraded dataset.

### **6.1 Clean the dataset**

In [14]:
if preprocess_image:
    create_or_clear_directory(os.path.join("..", "Dataset", "Preprocessed"))

    df = pd.DataFrame(columns=["Image", "label"])

    pipe = ImagePipeline(knn, extractor)

    for i, row in tqdm(
        val_deg.labels.iterrows(), total=len(val_deg.labels), file=sys.stdout
    ):
        path = os.path.join("..", "Dataset", "val_set_degraded", row["Image"])
        img = cv2.imread(path)
        post_img = pipe.preprocess(img, (20, 70))

        if not isinstance(post_img, np.ndarray) and post_img == -1:
            continue

        save_path = os.path.join("..", "Dataset", "Preprocessed", row["Image"])
        cv2.imwrite(save_path, post_img)

        df = pd.concat(
            [
                df,
                pd.DataFrame({"Image": row["Image"], "label": row["Label"]}, index=[0]),
            ],
            ignore_index=True,
        )

    df.to_csv(os.path.join("..", "Dataset", "val_deg_process.csv"), index=False)

  0%|          | 1/11994 [00:00<22:34,  8.85it/s]


TypeError: '>=' not supported between instances of 'str' and 'int'

### **6.2 k-NN Classifier**

In [ ]:
knn_pred_deg = knn.predict(x_val_deg)
print(f"Deg: {accuracy_score(y_val_deg, knn_pred_deg)}")

knn_score_deg = knn.predict_proba(x_val_deg)
print(f"Deg top 5: {top_k_accuracy_score(y_val_deg, knn_score_deg, k=5)}")

[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
[3, 3, 64, 23, 32, 2]
Deg: 0.015851137146795313


### **6.3 Random Forest Classifier**

In [ ]:
rf_pred_deg = rf.predict(x_val_deg)
print(accuracy_score(y_val_deg, rf_pred_deg))

rf_score_deg = rf.predict_proba(x_val_deg)
print(top_k_accuracy_score(y_val_deg, rf_score_deg, k=5))